<a href="https://colab.research.google.com/github/Moostafaaa/chromadb_Langchain/blob/main/chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  openai langchain sentence_transformers chromadb unstructured -q

In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset


dataset_name = 'cais/mmlu'
dataset = load_dataset(dataset_name)

In [ ]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.DataFrame(dataset['train'])

In [ ]:
df_train

,id,question,opa,opb,opc,opd,cop,choice_type,exp,subject_name,topic_name
0,e9ad821a-c438-4965-9f77-760819dfa155,Chronic urethral obstruction due to benign pri...,Hyperplasia,Hyperophy,Atrophy,Dyplasia,2,single,Chronic urethral obstruction because of urinar...,Anatomy,Urinary tract
1,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,Which vitamin is supplied from only animal sou...,Vitamin C,Vitamin B7,Vitamin B12,Vitamin D,2,single,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...,Biochemistry,Vitamins and Minerals
2,5c38bea6-787a-44a9-b2df-88f4218ab914,All of the following are surgical options for ...,Adjustable gastric banding,Biliopancreatic diversion,Duodenal Switch,Roux en Y Duodenal By pass,3,multi,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba...",Surgery,Surgical Treatment Obesity
3,cdeedb04-fbe9-432c-937c-d53ac24475de,Following endaerectomy on the right common car...,Central aery of the retina,Infraorbital aery,Lacrimal aery,Nasociliary aretry,0,multi,The central aery of the retina is a branch of ...,Ophthalmology,None
4,dc6794a3-b108-47c5-8b1b-3b4931577249,Growth hormone has its effect on growth through?,Directly,IG1-1,Thyroxine,Intranuclear receptors,1,single,"Ans. is 'b' i.e., IGI-1GH has two major functi...",Physiology,None
...,...,...,...,...,...,...,...,...,...,...,...
182817,04519631-61e3-4ef4-b2d6-c352cf349ded,Most common site for extra mammary Paget&;s di...,Vulva,Vagina,Penis,Anus,0,single,.It is superficial manifestation of an intradu...,Surgery,Urology
182818,fa6b40a1-459a-401c-96e7-fac5608e7744,Inferior Rib notching is seen in all except?,Coarctation of aoa,Classical blalock tausing operation,SVC obstruction,Neurofibromatosis,3,multi,Answer is D (Neurofibromatosis) Neurofibromato...,Medicine,None
182819,4ecebffe-72f6-4988-8d64-738e43ad6086,Which is false regarding cryptococcus neoformans?,Grows at 5degC and 37degC,It has 4 serotypes,Urease negative,Causes superficial skin infection,2,multi,"Ans. is 'c' i e., Urease negative Cryptococcus...",Microbiology,None
182820,7cf99676-f2b5-4f6d-9cf5-03ff7651afb3,Histopathological finding of gluten hypersensi...,Crypt hyperplasia,Increase in thickness of the mucosa,Distal intestine involvement,Villous hypertrophy,0,single,"Ans. is 'a' i.e., Crypt hyperplasia Histopatho...",Pathology,G.I.T.


In [ ]:
exp = df_train.exp

In [ ]:
exp.isnull().sum()

21953

In [ ]:
exp.dropna(inplace=True)

In [ ]:
ls = []
for txt in exp:
  ls.append(len(txt))

len(ls)


160869

In [ ]:
max(ls)

22525

In [ ]:
exp.to_csv("expfile.csv")

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='/content/expfile.csv')
data = loader.load()

In [ ]:
data[10]

Document(page_content=": 10\nexp: Nagler's reaction - when Clostridium perfringens is grown in media containing 6 % agar, 5% Fildes peptic digest of sheep blood and 20% human serum, with antitoxin spread on one half of plate, colonies in the other half without antitoxin will be surrounded by a zone of opacity. No opacity around the colonies on the half of plate with antitoxin, this is due to the specific neutralization of the alpha-toxin. This specific lecithinase effect, known as the Nagler's reaction. Reference: Textbook of Microbiology; Anathanarayan and paniker's; 10th edition; Page no: 263", metadata={'source': '/content/expfile.csv', 'row': 10})

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=800,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(data)
print(len(docs))

234047


In [ ]:
docs[10]

Document(page_content=': 6\nexp: Abnormal vascular pattern include punctation, mosaicism and atypical vessels.', metadata={'source': '/content/expfile.csv', 'row': 6})

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
db = Chroma.from_documents(docs[0:10000], embeddings)

In [ ]:
docs[250]

Document(page_content=': 195\nexp: The axial skeleton shapes the longitudinal axis of the human body. Coracoclavicular ligament suspends the scapula from the lower one third of the clavicle and forms a strong bond between them. The weight of the upper limb is transmitted to the axial skeleton through this ligament. A fracture of the clavicle, medial to the attachment of this ligament leads to drooping of upper limb. Ref: TB of Anatomy and Physiology PR Ashalatha, 1st Ed, Page 110', metadata={'source': '/content/expfile.csv', 'row': 174})

In [ ]:
query = "What is axial skeleton?"
matching_docs = db.similarity_search(query)



In [ ]:
matching_docs[0]

Document(page_content=': 195\nexp: The axial skeleton shapes the longitudinal axis of the human body. Coracoclavicular ligament suspends the scapula from the lower one third of the clavicle and forms a strong bond between them. The weight of the upper limb is transmitted to the axial skeleton through this ligament. A fracture of the clavicle, medial to the attachment of this ligament leads to drooping of upper limb. Ref: TB of Anatomy and Physiology PR Ashalatha, 1st Ed, Page 110', metadata={'row': 174, 'source': '/content/expfile.csv'})

In [ ]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs[0:50000], embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()

In [ ]:
/content/drive/MyDrive/VDB

In [ ]:
! cp /content/expfile.csv /content/drive/MyDrive/VDB

In [ ]:
!apt-get install rar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 341 kB of archives.
After this operation, 1,202 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 rar amd64 2:5.5.0-1.1 [341 kB]
Fetched 341 kB in 0s (990 kB/s)
Selecting previously unselected package rar.
(Reading database ... 120500 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1.1_amd64.deb ...
Unpacking rar (2:5.5.0-1.1) ...
Setting up rar (2:5.5.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!rar a -p "/content/drive/MyDrive/VDB/db.rar" "/content/chroma_db"


Enter password (will not be echoed): 

Reenter password: 


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/drive/MyDrive/VDB/db.rar

Adding    /content/chroma_db/chroma.sqlite3                                1%  2%  3%  5%  6%  7%  9% 10% 11% 13% 14% 15% 17% 18% 19% 21% 22% 23% 25% 26% 27% 28% 30%123
123
 31%